# ATTN: This script uses Google translate to detect job description language. Google translate will limit requests and take a very long time. Only run this script if redoing language detection.

# Read from scrapped data

In [14]:
import os
import sys
import importlib
from pathlib import Path
import numpy as np

mod = sys.modules[__name__]

code_dir = None
code_dir_name = 'Code'
unwanted_subdir_name = 'Analysis'

for _ in range(5):

    parent_path = str(Path.cwd().parents[_]).split('/')[-1]

    if (code_dir_name in parent_path) and (unwanted_subdir_name not in parent_path):

        code_dir = str(Path.cwd().parents[_])

        if code_dir is not None:
            break

sys.path.append(code_dir)
# %load_ext autoreload
# %autoreload 2


In [15]:
from setup_module.imports import *


#### Read paths

In [16]:
glob_paths = list(set(glob.glob(f'{scraped_data}Coding Material/*Folder/*/Job ID -*- Codebook (Automating Equity).xlsx')))


In [17]:
# 244 xlsx files
len(glob_paths)


244

: 

#### Use paths to open files, fix keywords, and drop unneeded columns

In [6]:
%%time
# columns
cols=['Sector', 
      'Sector Code', 
      'Gender', 
      'Age', 
      'Language', 
      'Dutch Requirement', 
      'English Requirement', 
      'Gender_Female', 
      'Gender_Mixed', 
      'Gender_Male', 
      'Age_Older', 
      'Age_Mixed', 
      'Age_Younger', 
      'Gender_Num', 
      'Age_Num', 
      '% Female', 
      '% Male', 
      '% Older', 
      '% Younger']

# Fix list catches all incorrect/faculty keyword search terms
fix_list = []

# Appended data catches all the fixed and cleaned dfs
appended_data = []

for glob_path in glob_paths:

    try:
        df_temp = pd.read_excel(glob_path).reset_index(drop=True)
    except ValueError:
        fix_list.append(glob_path)

    if len(df_temp) > 0 and isinstance(df_temp, pd.DataFrame):
        df_temp.reset_index(drop=True, inplace=True)
        df_temp.drop(columns=cols, axis='columns', inplace=True, errors='ignore')
        df_temp.drop(
        df_temp.columns[
                df_temp.columns.str.contains(
                    'unnamed|index|level', regex=True, case=False, flags=re.I
                )
            ],
            axis='columns',
            inplace=True,
            errors='ignore',
        )

        appended_data.append(df_temp.reset_index(drop=True))

# Concatonate list of dfs into one large df_manual
df_manual = pd.concat(appended_data, axis='index').reset_index(drop=True)

# Save df_manual to file
assert len(df_manual) > 0 and isinstance(df_manual, pd.DataFrame)
df_manual.to_pickle(f'{df_save_dir}df_manual_raw.pkl')
df_manual.to_csv(f'{df_save_dir}df_manual_raw.csv', index=False)


CPU times: user 5.63 s, sys: 153 ms, total: 5.78 s
Wall time: 6.12 s


In [7]:
# If we couldn't fix some keywords, we add them to list fix_list and write to file
if len(fix_list) != 0:
    print('Some keywords to fix!')
    with open(f'{data_dir}fix_list.txt', 'w') as f:
        json.dump(fix_list, f)


In [8]:
# List of dfs, len = 244
len(appended_data)


244

In [9]:
# Concatonate list of dfs into one large df_manual
df_manual = pd.concat(appended_data, axis='index').reset_index(drop=True)


In [10]:
# len = 12400
len(df_manual)


12400

In [ ]:
# Save df_manual to file
assert len(df_manual) > 0 and isinstance(df_manual, pd.DataFrame)
df_manual.to_pickle(f'{df_save_dir}df_manual_raw.pkl')
df_manual.to_csv(f'{df_save_dir}df_manual_raw.csv', index=False)


# Drop duplicated and missing data

### START HERE IF SOURCING FROM df_manual_RAW
### PLEASE SET CORRECT DIRECTORY PATHS BELOW

In [ ]:
import os
import sys
import importlib
from pathlib import Path
import numpy as np

mod = sys.modules[__name__]

code_dir = None
code_dir_name = 'Code'
unwanted_subdir_name = 'Analysis'

for _ in range(5):

    parent_path = str(Path.cwd().parents[_]).split('/')[-1]

    if (code_dir_name in parent_path) and (unwanted_subdir_name not in parent_path):

        code_dir = str(Path.cwd().parents[_])

        if code_dir is not None:
            break

sys.path.append(code_dir)
# %load_ext autoreload
# %autoreload 2


In [ ]:
from setup_module.imports import *

In [ ]:
df_manual = pd.read_pickle(f'{df_save_dir}df_manual_raw.pkl').reset_index(drop=True)


In [ ]:
# len = 12400
len(df_manual)


In [ ]:
df_manual.info()


In [ ]:
df_manual.columns

In [ ]:
# Clean columns
df_manual.columns = df_manual.columns.to_series().apply(lambda x: str(x).strip())


In [ ]:
# Remove columns 'Task_Mentioned', 'Task_Warmth', 'Task_Competence'
df_manual.drop(
    columns=['Task_Mentioned', 'Task_Warmth', 'Task_Competence'],
    axis='columns',
    inplace=True,
    errors='ignore'
)

In [ ]:
df_manual['Warmth'] = df_manual['Warmth'].astype(np.float64)
df_manual['Competence'] = df_manual['Competence'].astype(np.float64)


In [ ]:
df_manual.info()


In [ ]:
# Rename Sentence to 'Job Description spacy_sentencized'
df_manual.rename(
    columns = {
        'Sentence': 'Job Description spacy_sentencized'
    },
    inplace=True,
    errors='ignore'
)

In [ ]:
df_manual.columns


In [ ]:
# Drop NA
df_manual.dropna(axis='index', how='all', inplace=True)
df_manual.dropna(axis='columns', how='all', inplace=True)
df_manual.dropna(
    subset = ['Job Description spacy_sentencized', 'Warmth', 'Competence'],
    inplace=True
)


In [ ]:
# len = 12394
len(df_manual)


In [ ]:
# len = 133
df_manual.groupby(['Job ID'])['Job ID'].unique()


In [ ]:
# Drop duplicates on subset of 'Job ID' and 'Sentence'
df_manual.drop_duplicates(subset=['Job ID', 'Job Description spacy_sentencized'], keep='first', ignore_index=True, inplace=True)


In [ ]:
# len = 6400
len(df_manual)


In [ ]:
# Remove any rows with missing 'Job ID'
df_manual.drop(
    df_manual[
        (df_manual['Job ID'].isin(nan_list)) | 
        (df_manual['Job ID'].isnull()) | 
        (df_manual['Job ID'].isna())
    ].index, 
    axis='index',
    inplace=True,
    errors='ignore'
)


In [ ]:
# len = 6400
len(df_manual)


In [ ]:
# Save df_manual to file
assert len(df_manual) > 0 and isinstance(df_manual, pd.DataFrame)
df_manual.to_pickle(f'{df_save_dir}df_manual_raw_dropped.pkl')
df_manual.to_csv(f'{df_save_dir}df_manual_raw_dropped.csv', index=False)


# Add English and Dutch language requirement columns

### START HERE IF SOURCING FROM df_manual_RAW_DROPPED
### PLEASE SET CORRECT DIRECTORY PATHS BELOW


In [ ]:
import os
import sys
import importlib
from pathlib import Path
import numpy as np

mod = sys.modules[__name__]

code_dir = None
code_dir_name = 'Code'
unwanted_subdir_name = 'Analysis'

for _ in range(5):

    parent_path = str(Path.cwd().parents[_]).split('/')[-1]

    if (code_dir_name in parent_path) and (unwanted_subdir_name not in parent_path):

        code_dir = str(Path.cwd().parents[_])

        if code_dir is not None:
            break

sys.path.append(code_dir)
# %load_ext autoreload
# %autoreload 2


In [ ]:
from setup_module.imports import *

In [ ]:
df_manual = pd.read_pickle(f'{df_save_dir}df_manual_raw_dropped.pkl').reset_index(drop=True)


In [ ]:
# 6400
len(df_manual)


In [ ]:
df_manual.info()


In [ ]:
# Add language requirement column
# Use regex to find language requirement
dutch_requirement_pattern = r'[Ll]anguage: [Dd]utch|[Dd]utch [Pp]referred|[Dd]utch [Re]quired|[Dd]utch [Ll]anguage|[Pp]roficient in [Dd]utch|[Ss]peak [Dd]utch|[Kk]now [Dd]utch'
english_requirement_pattern = r'[Ll]anguage: [Ee]nglish|[Ee]nglish [Pp]referred|[Ee]nglish [Re]quired|[Ee]nglish [Ll]anguage|[Pp]roficient in [Ee]nglish|[Ss]peak [Ee]nglish|[Kk]now [Ee]nglish'

lang_requirements = {
    'Dutch Requirement': dutch_requirement_pattern, 'English Requirement': english_requirement_pattern
}

for lang_req, lang_req_pattern in lang_requirements.items():
    
    if lang_req in df_manual.columns:
        df_manual.drop(columns=[lang_req], inplace=True)
    df_manual[lang_req] = np.where(
        df_manual['Job Description'].str.contains(lang_req_pattern),
        'Yes',
        'No',
    )

assert len(df_manual) > 0 and isinstance(df_manual, pd.DataFrame)
df_manual.to_pickle(f'{df_save_dir}df_manual_raw_language_requirement.pkl')
df_manual.to_csv(f'{df_save_dir}df_manual_raw_english_requirement.csv', index=False)


In [ ]:
# Yes = 235
df_manual['Dutch Requirement'].value_counts()


In [ ]:
# Yes = 526
df_manual['English Requirement'].value_counts()

In [ ]:
assert len(df_manual) > 0 and isinstance(df_manual, pd.DataFrame)
df_manual.to_pickle(f'{df_save_dir}df_manual_raw_language_requirement.pkl')
df_manual.to_csv(f'{df_save_dir}df_manual_raw_language_requirement.csv', index=False)


# Add data from Sectors dataframe (see CBS directory under scrapped_data directory) and Categorical data


### START HERE IF SOURCING FROM df_manual_RAW_LANGUAGE_REQUIREMENT
### PLEASE SET CORRECT DIRECTORY PATHS BELOW


In [ ]:
import os
import sys
import importlib
from pathlib import Path
import numpy as np

mod = sys.modules[__name__]

code_dir = None
code_dir_name = 'Code'
unwanted_subdir_name = 'Analysis'

for _ in range(5):

    parent_path = str(Path.cwd().parents[_]).split('/')[-1]

    if (code_dir_name in parent_path) and (unwanted_subdir_name not in parent_path):

        code_dir = str(Path.cwd().parents[_])

        if code_dir is not None:
            break

sys.path.append(code_dir)
# %load_ext autoreload
# %autoreload 2


In [ ]:
from setup_module.imports import *

In [ ]:
def df_gender_age_info(df, ivs_all=None):
    if ivs_all is None:
        ivs_all = [
            'Gender',
            'Gender_Num',
            'Gender_Female',
            'Gender_Mixed',
            'Gender_Male',
            'Age',
            'Age_Num',
            'Age_Older',
            'Age_Mixed',
            'Age_Younger',
        ]
    # Print Info
    print('\nDF INFO:\n')
    df.info()

    for iv in ivs_all:
        try:
            counts = df[f"{iv}"].value_counts()
            percentages = df[f"{iv}"].value_counts(normalize=True).mul(100).round(1).astype(float)
            print('='*20)
            print(f'{iv}:')
            print('-'*20)
            print(f'{iv} Counts:\n{counts}')
            print('-'*20)
            print(f'{iv} Percentages:\n{percentages}')

            with contextlib.suppress(Exception):
                mean = df[f"{iv}"].mean().round(2).astype(float)
                sd = df[f"{iv}"].std().round(2).astype(float)
                print('-'*20)
                print(f'{iv} Mean: {mean}')
                print('-'*20)
                print(f'{iv} Standard Deviation: {sd}')

        except Exception:
            print(f'{iv} not available.')

    print('\n')


In [ ]:
df_manual = pd.read_pickle(f'{df_save_dir}df_manual_raw_language_requirement.pkl').reset_index(drop=True)


In [ ]:
df_manual.info()


In [ ]:
df_manual['Job ID'] = df_manual['Job ID'].apply(lambda x: str(x).lower().strip())


In [ ]:
df_jobs = pd.read_pickle(f'{df_save_dir}df_jobs_including_sector_genage_data.pkl').reset_index(drop=True)


In [ ]:
df_jobs.info()

In [ ]:
df_jobs['Job ID'] = df_jobs['Job ID'].apply(lambda x: str(x).lower().strip())


In [ ]:
df_jobs.columns


In [ ]:
df_jobs.drop(
    columns = [
        'Job Description', 'Rating', 'Employment Type',
        'Company URL', 'Job URL', 'Job Age', 'Job Age Number',
        'Collection Date', 'Data Row', 'Tracking ID', 'Job Date',
        'Type of ownership', 'Language', 'Dutch Requirement', 'English Requirement', 
    ],
    inplace=True,
    errors='ignore'
)

In [ ]:
df_jobs.columns


In [ ]:
# Add sector and categorical data from df_jobs
df_manual = df_manual.merge(df_jobs, on='Job ID', how='inner')


In [ ]:
len(df_manual)


In [ ]:
df_manual.info()


In [ ]:
df_manual.head()

#### Check if there is any missing sector data in the merged dataframe

In [ ]:
df_manual['Sector'].isna().sum()

In [ ]:
if df_manual['Sector'].isna().sum() != 0:
    print('Some search keywords did not match a sector. Fixing')
    print(set(df_manual['Search Keyword'].loc[df_manual['Sector'].isna()].to_list()))
    print(len(df_manual['Search Keyword'].loc[df_manual['Search Keyword'].isin(list(keyword_trans_dict.keys()))]))
    df_manual = fix_keywords(df_manual)
    print(set(df_manual['Search Keyword'].loc[df_manual['Sector'].isna()].to_list()))
    print(len(df_manual['Search Keyword'].loc[df_manual['Search Keyword'].isin(list(keyword_trans_dict.keys()))]))


In [ ]:
# Manual Job Ad info
df_gender_age_info(df_manual.groupby(['Job ID']).first())


In [ ]:
# Manual Job Sentence info
df_gender_age_info(df_manual)


In [ ]:
# Manual Job Sentence info
df_gender_age_info(df_manual, ivs_all=['Warmth', 'Competence'])


In [ ]:
if df_manual['Sector'].isna().sum() == 0:
    assert len(df_manual) > 0 and isinstance(df_manual, pd.DataFrame)
    df_manual.to_pickle(f'{df_save_dir}df_manual_including_sector_genage_data.pkl')
    df_manual.to_csv(f'{df_save_dir}df_manual_including_sector_genage_data.csv', index=False)
else:
    print(f"MISSING SECTOR DATA: COUNT {df_manual['Sector'].isna().sum()}")

# ATTN: This script should be run AFTER spacy sentence splitting is completed.


# Use spacy to tokenize sentences


### START HERE IF SOURCING FROM df_manual_SENTENCIZED
### PLEASE SET CORRECT DIRECTORY PATHS BELOW


In [ ]:
import os
import sys
import importlib
from pathlib import Path
import numpy as np

mod = sys.modules[__name__]

code_dir = None
code_dir_name = 'Code'
unwanted_subdir_name = 'Analysis'

for _ in range(5):

    parent_path = str(Path.cwd().parents[_]).split('/')[-1]

    if (code_dir_name in parent_path) and (unwanted_subdir_name not in parent_path):

        code_dir = str(Path.cwd().parents[_])

        if code_dir is not None:
            break

sys.path.append(code_dir)
# %load_ext autoreload
# %autoreload 2


In [ ]:
from setup_module.imports import *

In [ ]:
def get_word_num_and_frequency(row, text_col):

    row['Job Description num_words'] = len(str(row[f'{text_col}']).split())
    row['Job Description num_unique_words'] = len(set(str(row[f'{text_col}']).split()))
    row['Job Description num_chars'] = len(str(row[f'{text_col}']))
    row['Job Description num_punctuations'] = len([c for c in str(row[f'{text_col}']) if c in string.punctuation])

    return row


In [ ]:
df_manual = pd.read_pickle(f'{df_save_dir}df_manual_including_sector_genage_data.pkl').reset_index(drop=True)


In [ ]:
df_manual['Job Description spacy_sentencized_lower'] = df_manual['Job Description spacy_sentencized'].apply(
    lambda job_sentence: job_sentence.strip().lower()
)


In [ ]:
df_manual[['Job Description spacy_sentencized', 'Job Description spacy_sentencized_lower']].head()


In [ ]:
# Spacy tokenize
with open(f'{data_dir}punctuations.txt', 'rb') as f:
    custom_punct_chars = pickle.load(f)

df_manual['Job Description spacy_tokenized'] = df_manual[
    'Job Description spacy_sentencized'
].apply(
    lambda job_sentence: [
        str(token.text.strip().lower())
        for token in nlp.tokenizer(job_sentence)
        if len(token) != 0
        and not token.is_space
        and not token.is_stop
        and not token.is_punct
        and not token.is_bracket
        and not token.like_email
        and token.text not in custom_punct_chars
    ]
)

assert len(df_manual) > 0 and isinstance(df_manual, pd.DataFrame)
df_manual.to_pickle(f'{df_save_dir}df_manual_tokenized_spacy.pkl')
df_manual.to_csv(f'{df_save_dir}df_manual_tokenized_spacy.csv', index=False)


In [ ]:
df_manual['Job Description spacy_sentencized_cleaned'] = df_manual['Job Description spacy_tokenized'].str.join(' ')


In [ ]:
# Get sentence word frequencies
df_manual = df_manual.apply(
    lambda row: get_word_num_and_frequency(
        row=row, text_col='Job Description spacy_sentencized'
    ), 
    axis='columns',
    
)


In [ ]:
df_manual.columns


In [ ]:
assert len(df_manual) > 0 and isinstance(df_manual, pd.DataFrame)
df_manual.to_pickle(f'{df_save_dir}df_manual_tokenized_spacy.pkl')
df_manual.to_csv(f'{df_save_dir}df_manual_tokenized_spacy.csv', index=False)


# Use NLTK to tokenize sentences


### START HERE IF SOURCING FROM df_manual_TOKENIZED_SPACY
### PLEASE SET CORRECT DIRECTORY PATHS BELOW


In [ ]:
import os
import sys
import importlib
from pathlib import Path
import numpy as np

mod = sys.modules[__name__]

code_dir = None
code_dir_name = 'Code'
unwanted_subdir_name = 'Analysis'

for _ in range(5):

    parent_path = str(Path.cwd().parents[_]).split('/')[-1]

    if (code_dir_name in parent_path) and (unwanted_subdir_name not in parent_path):

        code_dir = str(Path.cwd().parents[_])

        if code_dir is not None:
            break

sys.path.append(code_dir)
# %load_ext autoreload
# %autoreload 2


In [ ]:
from setup_module.imports import *

In [ ]:
df_manual = pd.read_pickle(f'{df_save_dir}df_manual_tokenized_spacy.pkl').reset_index(drop=True)


In [ ]:
df_manual.info()


In [ ]:
%%time
# Tokenize with NLTK
df_manual['Job Description nltk_tokenized'] = df_manual['Job Description spacy_sentencized'].apply(
    lambda job_sentence: [
        str(token.strip().lower()) 
        for token in word_tokenize(job_sentence) 
        if len(token) != 0 
        and token != '...' 
        and not token.lower() in set(stopwords.words('english')) 
        and not token.lower() in list(string.punctuation) 
    ]
)

assert len(df_manual) > 0 and isinstance(df_manual, pd.DataFrame)
df_manual.to_pickle(f'{df_save_dir}df_manual_tokenized_spacy_nltk.pkl')
df_manual.to_csv(f'{df_save_dir}df_manual_tokenized_spacy_nltk.csv', index=False)


In [ ]:
df_manual['Job Description nltk_tokenized'].head()


In [ ]:
df_manual.info()


In [ ]:
assert len(df_manual) > 0 and isinstance(df_manual, pd.DataFrame)
df_manual.to_pickle(f'{df_save_dir}df_manual_tokenized_spacy_nltk.pkl')
df_manual.to_csv(f'{df_save_dir}df_manual_tokenized_spacy_nltk.csv', index=False)


# Use gensim to tokenize sentences


### START HERE IF SOURCING FROM df_manual_TOKENIZED_SPACY_NLTK
### PLEASE SET CORRECT DIRECTORY PATHS BELOW


In [ ]:
import os
import sys
import importlib
from pathlib import Path
import numpy as np

mod = sys.modules[__name__]

code_dir = None
code_dir_name = 'Code'
unwanted_subdir_name = 'Analysis'

for _ in range(5):

    parent_path = str(Path.cwd().parents[_]).split('/')[-1]

    if (code_dir_name in parent_path) and (unwanted_subdir_name not in parent_path):

        code_dir = str(Path.cwd().parents[_])

        if code_dir is not None:
            break

sys.path.append(code_dir)
# %load_ext autoreload
# %autoreload 2


In [ ]:
from setup_module.imports import *


In [ ]:
df_manual = pd.read_pickle(f'{df_save_dir}df_manual_tokenized_spacy_nltk.pkl').reset_index(drop=True)


In [ ]:
df_manual.info()


In [ ]:
%%time
df_manual['Job Description gensim_tokenized'] = df_manual['Job Description spacy_sentencized'].apply(
    lambda sentence: preprocess_string(re.sub(pattern, ' ', sentence.strip().lower()))
)

assert len(df_manual) > 0 and isinstance(df_manual, pd.DataFrame)
df_manual.to_pickle(f'{df_save_dir}df_manual_tokenized_spacy_nltk_gensim.pkl')
df_manual.to_csv(f'{df_save_dir}df_manual_tokenized_spacy_nltk_gensim.csv', index=False)


In [ ]:
df_manual['Job Description gensim_tokenized'].head()


In [ ]:
df_manual.info()


In [ ]:
assert len(df_manual) > 0 and isinstance(df_manual, pd.DataFrame)
df_manual.to_pickle(f'{df_save_dir}df_manual_tokenized_spacy_nltk_gensim.pkl')
df_manual.to_csv(f'{df_save_dir}df_manual_tokenized_spacy_nltk_gensim.csv', index=False)


# Use BERT to tokenize sentences


### START HERE IF SOURCING FROM df_manual_TOKENIZED_SPACY_NLTK_GENSIM
### PLEASE SET CORRECT DIRECTORY PATHS BELOW


In [ ]:
import os
import sys
import importlib
from pathlib import Path
import numpy as np

mod = sys.modules[__name__]

code_dir = None
code_dir_name = 'Code'
unwanted_subdir_name = 'Analysis'

for _ in range(5):

    parent_path = str(Path.cwd().parents[_]).split('/')[-1]

    if (code_dir_name in parent_path) and (unwanted_subdir_name not in parent_path):

        code_dir = str(Path.cwd().parents[_])

        if code_dir is not None:
            break

sys.path.append(code_dir)
# %load_ext autoreload
# %autoreload 2


In [ ]:
from setup_module.imports import *


In [ ]:
df_manual = pd.read_pickle(f'{df_save_dir}df_manual_tokenized_spacy_nltk_gensim.pkl').reset_index(drop=True)


In [ ]:
df_manual.info()


In [ ]:
%%time
df_manual['Job Description bert_encodings'] = df_manual['Job Description spacy_sentencized'].apply(
    lambda sentence: bert_tokenizer(str(sentence), truncation=True, padding=True)
)

df_manual['Job Description bert_tokenized'] = df_manual['Job Description spacy_sentencized'].apply(
    lambda sentence: bert_tokenizer.tokenize(str(sentence))
)

df_manual['Job Description bert_tokenized_to_id'] = df_manual['Job Description bert_tokenized'].apply(
    lambda sentence: bert_tokenizer.convert_tokens_to_ids(str(sentence))
)

assert len(df_manual) > 0 and isinstance(df_manual, pd.DataFrame)
df_manual.to_pickle(f'{df_save_dir}df_manual_tokenized_spacy_nltk_gensim_bert.pkl')
df_manual.to_csv(f'{df_save_dir}df_manual_tokenized_spacy_nltk_gensim_bert.csv', index=False)


In [ ]:
df_manual['Job Description bert_encodings'].head()


In [ ]:
df_manual['Job Description bert_tokenized'].head()


In [ ]:
df_manual['Job Description bert_tokenized_to_id'].head()

In [ ]:
assert len(df_manual) > 0 and isinstance(df_manual, pd.DataFrame)
df_manual.to_pickle(f'{df_save_dir}df_manual_tokenized_spacy_nltk_gensim_bert.pkl')
df_manual.to_csv(f'{df_save_dir}df_manual_tokenized_spacy_nltk_gensim_bert.csv', index=False)


# ATTN: This script should be run AFTER all tokenization (spacy, nltk, gensim, and BERT) completed.


# Use spacy to create Parts-Of-Speech (POS) tags, lemmas, and stems


### START HERE IF SOURCING FROM df_manual_TOKENIZED_SPACY_NLTK_GENSIM_BERT
### PLEASE SET CORRECT DIRECTORY PATHS BELOW


In [ ]:
import os
import sys
import importlib
from pathlib import Path
import numpy as np

mod = sys.modules[__name__]

code_dir = None
code_dir_name = 'Code'
unwanted_subdir_name = 'Analysis'

for _ in range(5):

    parent_path = str(Path.cwd().parents[_]).split('/')[-1]

    if (code_dir_name in parent_path) and (unwanted_subdir_name not in parent_path):

        code_dir = str(Path.cwd().parents[_])

        if code_dir is not None:
            break

sys.path.append(code_dir)
# %load_ext autoreload
# %autoreload 2


In [ ]:
from setup_module.imports import *


In [ ]:
df_manual = pd.read_pickle(f'{df_save_dir}df_manual_tokenized_spacy_nltk_gensim_bert.pkl').reset_index(drop=True)


In [ ]:
df_manual.info()


In [ ]:
# Load customer characters
with open(f'{data_dir}punctuations.txt', 'rb') as f:
    custom_punct_chars = pickle.load(f)

# POS tagging
df_manual['Job Description spacy_token_tags'] = df_manual[
    'Job Description spacy_sentencized'
].apply(
    lambda job_sentence: [
        (token.text.strip().lower(), token.tag_) for token in nlp(job_sentence)
    ]
)

# Lemmatization
df_manual['Job Description spacy_lemmas'] = df_manual['Job Description spacy_sentencized'].apply(
    lambda job_sentence: [
        token.lemma_.strip().lower()
        for token in nlp(job_sentence)
        if len(token) != 0 and not token.is_stop and not token.is_punct and token.text not in custom_punct_chars
    ]
)

# Stemming
df_manual['Job Description spacy_stems'] = df_manual['Job Description spacy_sentencized'].apply(
    lambda job_sentence: [
        stemmer.stem(token.text.strip().lower())
        for token in nlp(job_sentence)
        if len(token) != 0 and not token.is_stop and not token.is_punct and token.text not in custom_punct_chars
    ]
)

assert len(df_manual) > 0 and isinstance(df_manual, pd.DataFrame)
df_manual.to_pickle(f'{df_save_dir}df_manual_tags_lemmas_stems_spacy.pkl')
df_manual.to_csv(f'{df_save_dir}df_manual_tags_lemmas_stems_spacy.csv', index=False)


In [ ]:
df_manual.info()


In [ ]:
df_manual[
    [
        'Job Description spacy_token_tags',
        'Job Description spacy_lemmas',
        'Job Description spacy_stems'
    ]
].head()


In [ ]:
assert len(df_manual) > 0 and isinstance(df_manual, pd.DataFrame)
df_manual.to_pickle(f'{df_save_dir}df_manual_tags_lemmas_stems_spacy.pkl')
df_manual.to_csv(f'{df_save_dir}df_manual_tags_lemmas_stems_spacy.csv', index=False)


# Use NLTK to create Parts-Of-Speech (POS) tags, lemmas, and stems


### START HERE IF SOURCING FROM df_manual_TAGS_LEMMAS_STEMS_SPACY
### PLEASE SET CORRECT DIRECTORY PATHS BELOW


In [ ]:
import os
import sys
import importlib
from pathlib import Path
import numpy as np

mod = sys.modules[__name__]

code_dir = None
code_dir_name = 'Code'
unwanted_subdir_name = 'Analysis'

for _ in range(5):

    parent_path = str(Path.cwd().parents[_]).split('/')[-1]

    if (code_dir_name in parent_path) and (unwanted_subdir_name not in parent_path):

        code_dir = str(Path.cwd().parents[_])

        if code_dir is not None:
            break

sys.path.append(code_dir)
# %load_ext autoreload
# %autoreload 2


In [ ]:
from setup_module.imports import *


In [ ]:
def get_wordnet_pos(token):
    """Map POS tag to first character lemmatize() accepts"""
    tag = nltk.pos_tag([token])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}

    return tag_dict.get(tag, wordnet.NOUN)


In [ ]:
df_manual = pd.read_pickle(f'{df_save_dir}df_manual_tags_lemmas_stems_spacy.pkl').reset_index(drop=True)


In [ ]:
df_manual.info()


In [ ]:
# POS stagging
df_manual['Job Description nltk_token_tags'] = df_manual['Job Description spacy_tokenized'].apply(
    lambda token: pos_tag(token)
)

# Lemmatization
df_manual['Job Description nltk_lemmas'] = df_manual['Job Description spacy_tokenized'].apply(
    lambda tokens: [
        lemmatizer.lemmatize(
            token, get_wordnet_pos(
                unicodedata.normalize('NFKD', str(token.strip().lower())).encode('ascii', 'ignore').decode('utf-8', 'ignore')
            )
        )
        for token in tokens
    ]
)

# Stemming
df_manual['Job Description nltk_stems'] = df_manual['Job Description spacy_tokenized'].apply(
    lambda tokens: [
        stemmer.stem(
            unicodedata.normalize('NFKD', str(token.strip().lower())).encode('ascii', 'ignore').decode('utf-8', 'ignore')
        )
        for token in tokens
    ]
)

assert len(df_manual) > 0 and isinstance(df_manual, pd.DataFrame)
df_manual.to_pickle(f'{df_save_dir}df_manual_tags_lemmas_stems_spacy_nltk.pkl')
df_manual.to_csv(f'{df_save_dir}df_manual_tags_lemmas_stems_spacy_nltk.csv', index=False)


In [ ]:
df_manual.info()


In [ ]:
df_manual[['Job Description nltk_token_tags', 'Job Description nltk_lemmas', 'Job Description nltk_stems']].head()


In [ ]:
assert len(df_manual) > 0 and isinstance(df_manual, pd.DataFrame)
df_manual.to_pickle(f'{df_save_dir}df_manual_tags_lemmas_stems_spacy_nltk.pkl')
df_manual.to_csv(f'{df_save_dir}df_manual_tags_lemmas_stems_spacy_nltk.csv', index=False)


# Use BERT to create Parts-Of-Speech (POS) tags, lemmas, and stems


### START HERE IF SOURCING FROM df_manual_TAGS_LEMMAS_STEMS_SPACY_NLTK
### PLEASE SET CORRECT DIRECTORY PATHS BELOW


In [ ]:
# import os
# import sys
# import importlib
# from pathlib import Path
# import numpy as np

# mod = sys.modules[__name__]

# code_dir = None
# code_dir_name = 'Code'
# unwanted_subdir_name = 'Analysis'

# for _ in range(5):

#     parent_path = str(Path.cwd().parents[_]).split('/')[-1]

#     if (code_dir_name in parent_path) and (unwanted_subdir_name not in parent_path):

#         code_dir = str(Path.cwd().parents[_])

#         if code_dir is not None:
#             break

# sys.path.append(code_dir)
# # %load_ext autoreload
# # %autoreload 2


In [ ]:
# from setup_module.imports import *


In [ ]:
# df_manual = pd.read_pickle(f'{df_save_dir}df_manual_tags_lemmas_stems_spacy_nltk.pkl').reset_index(drop=True)



In [ ]:
# %%time
# bert_pos_model_name = 'QCRI/bert-base-multilingual-cased-pos-english'
# bert_pos_model = AutoModelForTokenClassification.from_pretrained(bert_pos_model_name)
# bert_pos_tagger = TokenClassificationPipeline(model=bert_pos_model, tokenizer=bert_tokenizer)

# df_manual['Job Description bert_token_tags_with_scores'] = df_manual['Job Description spacy_sentencized'].apply(
#     lambda sentence: [
#         (bert_pos_tag['word'], bert_pos_tag['entity'], bert_pos_tag['score'])
#         for i in range(len(sentence.split()))
#         for bert_pos_tag in bert_pos_tagger(sentence)
#     ]
# )

# df_manual['Job Description bert_token_tags'] = df_manual['Job Description bert_token_tags_with_scores'].apply(
#     lambda tag_list: [
#         [(tag_list[i][0], tag_list[i][1])]
#         for tag_tuple in tag_list
#         for i in range(len(tag_list))
#     ]
# )


# assert len(df_manual) > 0 and isinstance(df_manual, pd.DataFrame)
# df_manual.to_pickle(f'{df_save_dir}df_manual_tags_lemmas_stems_spacy_nltk_bert.pkl')
# df_manual.to_csv(f'{df_save_dir}df_manual_tags_lemmas_stems_spacy_nltk_bert.csv', index=False)


In [ ]:
# df_manual['Job Description bert_token_tags'].head()

In [ ]:
# assert len(df_manual) > 0 and isinstance(df_manual, pd.DataFrame)
# df_manual.to_pickle(f'{df_save_dir}df_manual_tags_lemmas_stems_spacy_nltk_bert.pkl')
# df_manual.to_csv(f'{df_save_dir}df_manual_tags_lemmas_stems_spacy_nltk_bert.csv', index=False)


# ATTN: This script should be run AFTER all POS tagging, lemmatization, and stemming (spacy and nltk) completed.
# If BERT POS tagging was done, change pkl file loading


### START HERE IF SOURCING FROM df_manual_TAGS_LEMMAS_STEMS_SPACY_NLTK
### IF BERT POS TAGGING WAS DONE, SOURCING FROM df_manual_TAGS_LEMMAS_STEMS_SPACY_NLTK_BERT
### PLEASE SET CORRECT DIRECTORY PATHS BELOW


# Use spacy to create bi and trigrams


In [ ]:
import os
import sys
import importlib
from pathlib import Path
import numpy as np

mod = sys.modules[__name__]

code_dir = None
code_dir_name = 'Code'
unwanted_subdir_name = 'Analysis'

for _ in range(5):

    parent_path = str(Path.cwd().parents[_]).split('/')[-1]

    if (code_dir_name in parent_path) and (unwanted_subdir_name not in parent_path):

        code_dir = str(Path.cwd().parents[_])

        if code_dir is not None:
            break

sys.path.append(code_dir)
# %load_ext autoreload
# %autoreload 2


In [ ]:
from setup_module.imports import *


In [ ]:
def spacy_make_ngrams(sentence, matcher, gram_type):

    doc = nlp(sentence)
    matches = matcher(doc)
    matches_list = []

    for idx in range(len(matches)):
        for match_id, start, end in matches:
            if nlp.vocab.strings[match_id].split('_')[0] == gram_type:
                match = doc[matches[idx][1]: matches[idx][2]].text
                matches_list.append(match.lower())
    
    return list(set(matches_list))


In [ ]:
df_manual = pd.read_pickle(f'{df_save_dir}df_manual_tags_lemmas_stems_spacy_nltk.pkl').reset_index(drop=True)


In [ ]:
df_manual['Job Description spacy_1grams_original_list'] = df_manual['Job Description spacy_tokenized']
df_manual['Job Description spacy_1grams'] = df_manual['Job Description spacy_tokenized'].apply(
    lambda tokens: [
        tuple(token.split())
        for token in tokens
    ]
)


In [ ]:
# Spacy bi and trigrams
matcher = Matcher(nlp.vocab)

bigram_rules = [
    ['NOUN', 'VERB'],
    ['VERB', 'NOUN'],
    ['ADJ', 'NOUN'],
    ['ADJ', 'PROPN'],
    # more rules here...
]

trigram_rules = [
    ['VERB', 'ADJ', 'NOUN'],
    ['NOUN', 'VERB', 'ADV'],
    ['NOUN', 'ADP', 'NOUN'],
    # more rules here...
]

patters_dict = {
    'bigram_patterns': [[{'POS': i} for i in j] for j in bigram_rules],
    'trigram_patterns': [[{'POS': i} for i in j] for j in trigram_rules],
}

ngram_dict = {
    'bigram': 2,
    'trigram': 3,
}

for ngram_name, ngram_num in ngram_dict.items():
    
    
    matcher.add(f'{ngram_name}_patterns', patters_dict[f'{ngram_name}_patterns'])

    df_manual[f'Job Description spacy_{str(ngram_num)}grams_original_list'] = df_manual['Job Description spacy_sentencized'].apply(
        lambda sentence: 
            [
                '_'.join(ngram_.split())
                for ngram_ in spacy_make_ngrams(sentence, matcher, ngram_name)
            ]
    )
    
    df_manual[f'Job Description spacy_{str(ngram_num)}grams'] = df_manual['Job Description spacy_sentencized'].apply(
        lambda sentence: 
            [
                tuple(ngram_.split())
                for ngram_ in spacy_make_ngrams(sentence, matcher, ngram_name)
            ]
    )

    df_manual[f'Job Description spacy_{str(ngram_num)}grams_in_sent'] = df_manual['Job Description spacy_sentencized'].str.lower().replace(
        regex = {
            re.escape(' '.join(ngram_.split('_'))): re.escape(ngram_)
            for ngrams_list in df_manual[f'Job Description spacy_{str(ngram_num)}grams_original_list']
            for ngram_ in ngrams_list
            if '_' in ngram_
        }
    )
    
    if f'{ngram_name}_patterns' in matcher:
        matcher.remove(f'{ngram_name}_patterns')
    assert f'{ngram_name}_patterns' not in matcher


In [ ]:
# Spacy Allgrams
df_manual['Job Description spacy_123grams_original_list'] = df_manual['Job Description spacy_tokenized'] + df_manual['Job Description spacy_2grams_original_list'] + df_manual['Job Description spacy_3grams_original_list']
df_manual['Job Description spacy_123grams'] = df_manual['Job Description spacy_1grams'] + df_manual['Job Description spacy_2grams'] + df_manual['Job Description spacy_3grams']
df_manual['Job Description spacy_123grams_in_sent'] = (
    df_manual['Job Description spacy_sentencized']
    .str.lower()
    .replace(
        regex={
            re.escape(' '.join(ngram_.split('_'))): re.escape(ngram_)
            for ngrams_list in df_manual[
                'Job Description spacy_123grams_original_list'
            ]
            for ngram_ in ngrams_list
            if '_' in ngram_
        }
    )
)


In [ ]:
assert len(df_manual) > 0 and isinstance(df_manual, pd.DataFrame)
df_manual.to_pickle(f'{df_save_dir}df_manual_ngrams_spacy.pkl')
df_manual.to_csv(f'{df_save_dir}df_manual_ngrams_spacy.csv', index=False)


# Use NLTK to create bi and trigrams


### START HERE IF SOURCING FROM df_manual_NGRAMS_SPACY
### PLEASE SET CORRECT DIRECTORY PATHS BELOW


In [ ]:
import os
import sys
import importlib
from pathlib import Path
import numpy as np

mod = sys.modules[__name__]

code_dir = None
code_dir_name = 'Code'
unwanted_subdir_name = 'Analysis'

for _ in range(5):

    parent_path = str(Path.cwd().parents[_]).split('/')[-1]

    if (code_dir_name in parent_path) and (unwanted_subdir_name not in parent_path):

        code_dir = str(Path.cwd().parents[_])

        if code_dir is not None:
            break

sys.path.append(code_dir)
# %load_ext autoreload
# %autoreload 2


In [ ]:
from setup_module.imports import *


In [ ]:
df_manual = pd.read_pickle(f'{df_save_dir}df_manual_ngrams_spacy.pkl').reset_index(drop=True)


In [ ]:
df_manual['Job Description nltk_1grams_original_list'] = df_manual['Job Description nltk_tokenized']
df_manual['Job Description nltk_1grams'] = df_manual['Job Description nltk_tokenized'].apply(
    lambda tokens: [
        tuple(token.split())
        for token in tokens
    ]
)


In [ ]:
# NLTK bi and trigrams
ngram_dict = {
    'bigram': 2,
    'trigram': 3
}

for ngram_name, ngram_num in ngram_dict.items():

    df_manual[f'Job Description nltk_{str(ngram_num)}grams_original_list'] = df_manual['Job Description nltk_tokenized'].apply(
        lambda tokens:
            list(
                '_'.join(ngram_list)
                for ngram_list in ngrams(tokens, ngram_num)
            )
    )

    df_manual[f'Job Description nltk_{str(ngram_num)}grams'] = df_manual['Job Description nltk_tokenized'].apply(
        lambda tokens: list(ngrams(tokens, ngram_num))
    )

    df_manual[f'Job Description nltk_{str(ngram_num)}grams_in_sent'] = df_manual['Job Description spacy_sentencized'].str.lower().replace(
        regex = {
            re.escape(' '.join(ngram_.split('_'))): re.escape(ngram_)
            for ngrams_list in df_manual[f'Job Description nltk_{str(ngram_num)}grams_original_list']
            for ngram_ in ngrams_list
            if '_' in ngram_
        }
    )


In [ ]:
# NLTK Allgrams
df_manual['Job Description nltk_123grams_original_list'] = (
    df_manual['Job Description nltk_tokenized']
    + df_manual['Job Description nltk_2grams_original_list']
    + df_manual['Job Description nltk_3grams_original_list']
)
df_manual['Job Description nltk_123grams'] = (
    df_manual['Job Description nltk_1grams']
    + df_manual['Job Description nltk_2grams']
    + df_manual['Job Description nltk_3grams']
)
df_manual['Job Description nltk_123grams_in_sent'] = (
    df_manual['Job Description spacy_sentencized']
    .str.lower()
    .replace(
        regex={
            re.escape(' '.join(ngram_.split('_'))): re.escape(ngram_)
            for ngrams_list in df_manual[
                'Job Description nltk_123grams_original_list'
            ]
            for ngram_ in ngrams_list
            if '_' in ngram_
        }
    )
)


In [ ]:
assert len(df_manual) > 0 and isinstance(df_manual, pd.DataFrame)
df_manual.to_pickle(f'{df_save_dir}df_manual_ngrams_spacy_nltk.pkl')
df_manual.to_csv(f'{df_save_dir}df_manual_ngrams_spacy_nltk.csv', index=False)


# Use Gensim to create bi and trigrams


### START HERE IF SOURCING FROM df_manual_NGRAMS_SPACY_NLTK
### PLEASE SET CORRECT DIRECTORY PATHS BELOW


In [ ]:
import os
import sys
import importlib
from pathlib import Path
import numpy as np

mod = sys.modules[__name__]

code_dir = None
code_dir_name = 'Code'
unwanted_subdir_name = 'Analysis'

for _ in range(5):

    parent_path = str(Path.cwd().parents[_]).split('/')[-1]

    if (code_dir_name in parent_path) and (unwanted_subdir_name not in parent_path):

        code_dir = str(Path.cwd().parents[_])

        if code_dir is not None:
            break

sys.path.append(code_dir)
# %load_ext autoreload
# %autoreload 2


In [ ]:
from setup_module.imports import *


In [ ]:
df_manual = pd.read_pickle(f'{df_save_dir}df_manual_ngrams_spacy_nltk.pkl').reset_index(drop=True)


In [ ]:
df_manual['Job Description gensim_1grams_original_list'] = df_manual['Job Description gensim_tokenized']
df_manual['Job Description gensim_1grams'] = df_manual['Job Description gensim_tokenized'].apply(
    lambda tokens: [
        tuple(token.split())
        for token in tokens
    ]
)


In [ ]:
# Gensim bi and trigrams
pattern = r'[\n]+|[,]{2,}|[|]{2,}|[\n\r]+|(?<=[a-z]\.)(?=\s*[A-Z])|(?=\:+[A-Z])'

# Gensim Bigrams
bigram = Phraser(Phrases(df_manual['Job Description gensim_tokenized'], connector_words=ENGLISH_CONNECTOR_WORDS, min_count=1, threshold=1))
df_manual['Job Description gensim_2grams_original_list_all'] = bigram[df_manual['Job Description gensim_tokenized']]
df_manual['Job Description gensim_2grams_original_list'] = df_manual['Job Description gensim_2grams_original_list_all'].apply(
    lambda ngrams_list: [
        ngram_
        for ngram_ in ngrams_list
        if len(re.findall('[a-zA-Z]*\_[a-zA-Z]*', ngram_)) != 0
    ]
)
df_manual['Job Description gensim_2grams'] = df_manual['Job Description gensim_2grams_original_list'].apply(
    lambda ngrams: [
        tuple(ngram.split('_'))
        for ngram in ngrams
        if '_' in ngram
    ]
)
df_manual['Job Description gensim_2grams_in_sent'] = (
    df_manual['Job Description spacy_sentencized']
    .str.lower()
    .apply(
        lambda sentence: ' '.join(
            preprocess_string(re.sub(pattern, ' ', sentence.strip().lower()))
        )
    )
    .replace(
        regex={
            re.escape(' '.join(ngram_.split('_'))): re.escape(ngram_)
            for ngrams_list in df_manual[
                'Job Description gensim_2grams_original_list'
            ]
            for ngram_ in ngrams_list
            if '_' in ngram_
        }
    )
)

# Gensim Trigrams
trigram = Phraser(Phrases(df_manual['Job Description gensim_2grams_original_list_all'], connector_words=ENGLISH_CONNECTOR_WORDS, min_count=1, threshold=1))
df_manual['Job Description gensim_3grams_original_list_all'] = trigram[df_manual['Job Description gensim_2grams_original_list_all']]
df_manual['Job Description gensim_3grams_original_list'] = df_manual['Job Description gensim_3grams_original_list_all'].apply(
    lambda ngrams_list: [
        ngram_
        for ngram_ in ngrams_list
        if len(re.findall('[a-zA-Z]*\_[a-zA-Z]*\_[a-zA-Z]*', ngram_)) != 0
    ]
)
df_manual['Job Description gensim_3grams'] = df_manual['Job Description gensim_3grams_original_list'].apply(
    lambda ngrams: [
        tuple(ngram.split('_'))
        for ngram in ngrams
        if '_' in ngram
    ]
)
df_manual['Job Description gensim_3grams_in_sent'] = (
    df_manual['Job Description spacy_sentencized']
    .str.lower()
    .apply(
        lambda sentence: ' '.join(
            preprocess_string(re.sub(pattern, ' ', sentence.strip().lower()))
        )
    )
    .replace(
        regex={
            re.escape(' '.join(ngram_.split('_'))): re.escape(ngram_)
            for ngrams_list in df_manual[
                'Job Description gensim_3grams_original_list'
            ]
            for ngram_ in ngrams_list
            if '_' in ngram_
        }
    )
)


In [ ]:
# Gensim Allgrams
pattern = r'[\n]+|[,]{2,}|[|]{2,}|[\n\r]+|(?<=[a-z]\.)(?=\s*[A-Z])|(?=\:+[A-Z])'

df_manual['Job Description gensim_123grams_original_list'] = (
    df_manual['Job Description gensim_tokenized']
    + df_manual['Job Description gensim_2grams_original_list']
    + df_manual['Job Description gensim_3grams_original_list']
)
df_manual['Job Description gensim_123grams'] = (
    df_manual['Job Description gensim_1grams']
    + df_manual['Job Description gensim_2grams']
    + df_manual['Job Description gensim_3grams']
)
df_manual['Job Description gensim_123grams_in_sent'] = (
    df_manual['Job Description spacy_sentencized']
    .str.lower()
    .apply(
        lambda sentence: ' '.join(
            preprocess_string(re.sub(pattern, ' ', sentence.strip().lower()))
        )
    )
    .replace(
        regex={
            re.escape(' '.join(ngram_.split('_'))): re.escape(ngram_)
            for ngrams_list in df_manual[
                'Job Description gensim_123grams_original_list'
            ]
            for ngram_ in ngrams_list
            if '_' in ngram_
        }
    )
)


In [ ]:
assert len(df_manual) > 0 and isinstance(df_manual, pd.DataFrame)
df_manual.to_pickle(f'{df_save_dir}df_manual_ngrams_spacy_nltk_gensim.pkl')
df_manual.to_csv(f'{df_save_dir}df_manual_ngrams_spacy_nltk_gensim.csv', index=False)


# Create word frequencies for uni, bi, and trigrams


### START HERE IF SOURCING FROM df_manual_NGRAMS_SPACY_NLTK_GENSIM
### PLEASE SET CORRECT DIRECTORY PATHS BELOW


In [ ]:
import os
import sys
import importlib
from pathlib import Path
import numpy as np

mod = sys.modules[__name__]

code_dir = None
code_dir_name = 'Code'
unwanted_subdir_name = 'Analysis'

for _ in range(5):

    parent_path = str(Path.cwd().parents[_]).split('/')[-1]

    if (code_dir_name in parent_path) and (unwanted_subdir_name not in parent_path):

        code_dir = str(Path.cwd().parents[_])

        if code_dir is not None:
            break

sys.path.append(code_dir)
# %load_ext autoreload
# %autoreload 2


In [ ]:
from setup_module.imports import *


In [ ]:
def get_abs_frequency(row, text_col, ngram_num, embedding_library):

    abs_word_freq = defaultdict(int)
    for word in row[f'Job Description {embedding_library}_{ngram_num}grams_original_list']:
        abs_word_freq[word] += 1

        abs_wtd_df = (
            pd.DataFrame.from_dict(abs_word_freq, orient='index')
            .rename(columns={0: 'abs_word_freq'})
            .sort_values(by=['abs_word_freq'], ascending=False)
            )
        abs_wtd_df.insert(1, 'abs_word_perc', value=abs_wtd_df['abs_word_freq'] / abs_wtd_df['abs_word_freq'].sum())
        abs_wtd_df.insert(2, 'abs_word_perc_cum', abs_wtd_df['abs_word_perc'].cumsum())

        row[f'Job Description {embedding_library}_{ngram_num}grams_abs_word_freq'] = str(abs_wtd_df['abs_word_freq'].to_dict())
        row[f'Job Description {embedding_library}_{ngram_num}grams_abs_word_perc'] = str(abs_wtd_df['abs_word_perc'].to_dict())
        row[f'Job Description {embedding_library}_{ngram_num}grams_abs_word_perc_cum'] = str(abs_wtd_df['abs_word_perc_cum'].to_dict())

    return row


In [ ]:
df_manual = pd.read_pickle(f'{df_save_dir}df_manual_ngrams_spacy_nltk_gensim.pkl').reset_index(drop=True)


In [ ]:
ngrams_list=[1, 2, 3, 123]
embedding_libraries_list = ['spacy', 'nltk', 'gensim']

for embedding_library, ngram_num in itertools.product(embedding_libraries_list, ngrams_list):
    df_manual = df_manual.apply(lambda row: get_abs_frequency(row=row, text_col='Job Description spacy_tokenized', ngram_num=ngram_num, embedding_library=embedding_library), axis='columns')


In [ ]:
assert len(df_manual) > 0 and isinstance(df_manual, pd.DataFrame)
df_manual.to_pickle(f'{df_save_dir}df_manual_ngrams_frequency.pkl')
df_manual.to_csv(f'{df_save_dir}df_manual_ngrams_frequency.csv', index=False)


# Create BoW dictionary, corpus, and tfidf matrix for uni, bi, and trigrams


### START HERE IF SOURCING FROM df_manual_NGRAMS_FREQUENCY
### PLEASE SET CORRECT DIRECTORY PATHS BELOW


In [ ]:
import os
import sys
import importlib
from pathlib import Path
import numpy as np

mod = sys.modules[__name__]

code_dir = None
code_dir_name = 'Code'
unwanted_subdir_name = 'Analysis'

for _ in range(5):

    parent_path = str(Path.cwd().parents[_]).split('/')[-1]

    if (code_dir_name in parent_path) and (unwanted_subdir_name not in parent_path):

        code_dir = str(Path.cwd().parents[_])

        if code_dir is not None:
            break

sys.path.append(code_dir)
# %load_ext autoreload
# %autoreload 2


In [ ]:
from setup_module.imports import *


In [ ]:
def get_corpus_and_dictionary(row, ngram_num, embedding_library):
    
    ngrams_original_list = row[f'Job Description {embedding_library}_{ngram_num}grams_original_list']
    dictionary = Dictionary([ngrams_original_list])
    BoW_corpus = [dictionary.doc2bow(ngrams_original_list)]
    tfidf = TfidfModel(BoW_corpus, smartirs='ntc')
    tfidf_matrix = [tfidf[doc] for doc in BoW_corpus]

    row[f'Job Description {embedding_library}_{ngram_num}grams_dictionary'] = dictionary
    row[f'Job Description {embedding_library}_{ngram_num}grams_BoW_corpus'] = BoW_corpus
    row[f'Job Description {embedding_library}_{ngram_num}grams_tfidf'] = tfidf
    row[f'Job Description {embedding_library}_{ngram_num}grams_tfidf_matrix'] = tfidf_matrix
    
    return row


In [ ]:
df_manual = pd.read_pickle(f'{df_save_dir}df_manual_ngrams_frequency.pkl').reset_index(drop=True)


In [ ]:
ngrams_list=[1, 2, 3, 123]
embedding_libraries_list = ['spacy', 'nltk', 'gensim']

for embedding_library, ngram_num in itertools.product(embedding_libraries_list, ngrams_list):
    df_manual = df_manual.apply(
        lambda row: get_corpus_and_dictionary(
            row=row, ngram_num=ngram_num, embedding_library=embedding_library
        ),
        axis='columns'
    )

assert len(df_manual) > 0 and isinstance(df_manual, pd.DataFrame)
df_manual.to_pickle(f'{df_save_dir}df_manual_ngrams_frequency.pkl')
df_manual.to_csv(f'{df_save_dir}df_manual_ngrams_BoW.csv', index=False)


In [ ]:
df_manual.columns


In [ ]:
assert len(df_manual) > 0 and isinstance(df_manual, pd.DataFrame)
df_manual.to_pickle(f'{df_save_dir}df_manual_ngrams_BoW.pkl')
df_manual.to_csv(f'{df_save_dir}df_manual_ngrams_BoW.csv', index=False)


# ATTN: This script should be run AFTER all bi and trigrams (spacy, nltk, and gensim) completed.


# Use spacy and nltk for sentiment scoring


### START HERE IF SOURCING FROM df_manual_NGRAMS_BOW
### PLEASE SET CORRECT DIRECTORY PATHS BELOW


In [ ]:
import os
import sys
import importlib
from pathlib import Path
import numpy as np

mod = sys.modules[__name__]

code_dir = None
code_dir_name = 'Code'
unwanted_subdir_name = 'Analysis'

for _ in range(5):

    parent_path = str(Path.cwd().parents[_]).split('/')[-1]

    if (code_dir_name in parent_path) and (unwanted_subdir_name not in parent_path):

        code_dir = str(Path.cwd().parents[_])

        if code_dir is not None:
            break

sys.path.append(code_dir)
# %load_ext autoreload
# %autoreload 2


In [ ]:
from setup_module.imports import *


In [ ]:
df_manual = pd.read_pickle(f'{df_save_dir}df_manual_ngrams_BoW.pkl').reset_index(drop=True)


In [ ]:
# Spacy sentiment
if 'spacytextblob' not in nlp.pipe_names:
    nlp.add_pipe('spacytextblob')

df_manual['Job Description spacy_sentiment'] = df_manual['Job Description spacy_sentencized'].apply(
    lambda sentence: float(nlp(sentence)._.blob.polarity)
    if isinstance(sentence, str) else np.nan
)


In [ ]:
# NLTK sentiment
df_manual['Job Description nltk_sentiment'] = df_manual['Job Description spacy_sentencized'].apply(
    lambda sentence: float(sentim_analyzer.polarity_scores(sentence)['compound'])
    if isinstance(sentence, str) else np.nan
)


In [ ]:
assert len(df_manual) > 0 and isinstance(df_manual, pd.DataFrame)
df_manual.to_pickle(f'{df_save_dir}df_manual_sentiment_spacy_nltk.pkl')
df_manual.to_csv(f'{df_save_dir}df_manual_sentiment_spacy_nltk.csv', index=False)


# ATTN: This script should be run AFTER all sentiment scoring (spacy and nltk) completed.


### START HERE IF SOURCING FROM df_manual_SENTIMENT_SPACY_NLTK
### PLEASE SET CORRECT DIRECTORY PATHS BELOW


# Word2Vec and FastText embeddings


In [ ]:
import os
import sys
import importlib
from pathlib import Path
import numpy as np

mod = sys.modules[__name__]

code_dir = None
code_dir_name = 'Code'
unwanted_subdir_name = 'Analysis'

for _ in range(5):

    parent_path = str(Path.cwd().parents[_]).split('/')[-1]

    if (code_dir_name in parent_path) and (unwanted_subdir_name not in parent_path):

        code_dir = str(Path.cwd().parents[_])

        if code_dir is not None:
            break

sys.path.append(code_dir)
# %load_ext autoreload
# %autoreload 2


In [ ]:
from setup_module.imports import *


In [ ]:
def build_train_word2vec(df, ngram_number, embedding_library, size = 300, words=None, t = time.time(), cores = multiprocessing.cpu_count()):
    if words is None:
        words = [
            'she',
            'he',
            'support',
            'leader',
            'management',
            'team',
            'business',
            'customer',
            'risk',
            'build',
            'computer',
            'programmer',
        ]
    sentences = df[f'Job Description {embedding_library}_{ngram_number}grams_original_list'].values

    w2v_model = Word2Vec(
        sentences=sentences,
        vector_size=size,
        min_count=0,
        window=2,
        sample=6e-5,
        alpha=0.03,
        min_alpha=0.0007,
        negative=20,
        workers=cores - 1,
        sg = 1,
    )

    w2v_model.build_vocab(sentences, progress_per=10000)
    print(f'Time to train the model for {size}: {round((time.time() - t) / 60, 2)} mins')

    w2v_model.train(
        sentences,
        total_examples=w2v_model.corpus_count,
        epochs=30,
        report_delay=1,
    )

    print(f'Time to build w2v_vocab for {size}: {round((time.time() - t) / 60, 2)} mins')
    w2v_vocab = list(w2v_model.wv.index_to_key)

    print(f'Checking words form list of length {len(words)}')
    print(f'WORDS LIST: {words}')

    for word in words:
        print(f'Checking word:\n{word.upper()}:')
        try:
#             print(f'Word2Vec {size}: {w2v_model.wv[word]}')
            print(f'Length of {size} model vobal: {len(w2v_vocab)}')
            print(f'{size} - Positive most similar to {word}: {w2v_model.wv.most_similar(positive=word, topn=5)}')
            print(f'{size} - Negative most similar to {word}: {w2v_model.wv.most_similar(negative=word, topn=5)}')

        except KeyError as e:
            print(e)

    return w2v_vocab, w2v_model

def word2vec_embeddings(sentences, w2v_vocab, w2v_model, size=300):

    sentences = [word for word in sentences if word in w2v_vocab]

    return (
        np.mean(w2v_model.wv[sentences], axis=0)
        if sentences
        else np.zeros(size)
    )



In [ ]:
def build_train_fasttext(df, ngram_number, embedding_library, size = 300, words=None, t = time.time(), cores = multiprocessing.cpu_count()):
    if words is None:
        words = [
            'she',
            'he',
            'support',
            'leader',
            'management',
            'team',
            'business',
            'customer',
            'risk',
            'build',
            'computer',
            'programmer',
        ]
    sentences = df[f'Job Description {embedding_library}_{ngram_number}grams_original_list'].values

    ft_model = FastText(
        sentences=sentences,
        vector_size=size,
        min_count=0,
        window=2,
        sample=6e-5,
        alpha=0.03,
        min_alpha=0.0007,
        negative=20,
        workers=cores - 1,
        sg = 1,
    )

    ft_model.build_vocab(sentences, progress_per=10000)
    print(f'Time to train the model for {size}: {round((time.time() - t) / 60, 2)} mins')

    ft_model.train(
        sentences,
        total_examples=ft_model.corpus_count,
        epochs=30,
        report_delay=1,
    )

    print(f'Time to build vocab for {size}: {round((time.time() - t) / 60, 2)} mins')
    ft_vocab = list(ft_model.wv.index_to_key)

    print(f'Checking words form list of length {len(words)}')
    print(f'WORDS LIST: {words}')

    for word in words:
        print(f'Checking word:\n{word.upper()}:')
        try:
#             print(f'FastText {size}: {ft_model_300.wv[word]}')
            print(f'Length of {size} model vobal: {len(ft_vocab)}')
            print(f'{size} - Positive most similar to {word}: {ft_model.wv.most_similar(positive=word, topn=5)}')
            print(f'{size} - Negative most similar to {word}: {ft_model.wv.most_similar(negative=word, topn=5)}')

        except KeyError as e:
            print(e)

    return ft_vocab, ft_model

def fasttext_embeddings(sentences, ft_vocab, ft_model, size=300):

    sentences = [word for word in sentences if word in ft_vocab]

    return np.mean(ft_model.wv[sentences], axis=0) if sentences else np.zeros(size)


In [ ]:
def get_glove(glove_file = f'{llm_path}/gensim/glove/glove.840B.300d.txt'):
    embeddings_index = {}
    with open(glove_file, 'r', encoding='utf8') as glove:

        for line in glove:
            values = line.split()
            word = values[0]

            with contextlib.suppress(ValueError):
                coefs = np.asarray(values[1:], dtype='float32')
                embeddings_index[word] = coefs
    print(f'Found {len(embeddings_index)} word vectors.')

    return embeddings_index


In [ ]:
def sent2vec(sentences, embeddings_index=None, external_glove=True, extra_preprocessing_enabled=False):

    if external_glove is False and embeddings_index is None:
        embeddings_index= get_glove()

    if extra_preprocessing_enabled is False:
        words = sentences

    elif extra_preprocessing_enabled is True:
        stop_words = set(sw.words('english'))
        words = str(sentences).lower()
        words = word_tokenize(words)
        words = [w for w in words if (w not in stop_words) and (w.isalpha())]

    M = []

    try:
        for w in words:
            try:
                M.append(embeddings_index[w])
            except Exception:
                continue

        M = np.array(M)
        v = M.sum(axis='index')
        return np.zeros(300) if type(v) != np.ndarray else v / np.sqrt((v ** 2).sum())

    except Exception:
        return np.zeros(300)


In [ ]:
df_manual = pd.read_pickle(f'{df_save_dir}df_manual_sentiment_spacy_nltk.pkl').reset_index(drop=True)


In [ ]:
embedding_models_dict = {
    'w2v': [build_train_word2vec, word2vec_embeddings, Word2Vec],
    'ft': [build_train_fasttext, fasttext_embeddings, FastText],
}


In [ ]:
ngrams_list=[1, 2, 3, 123]
embedding_libraries_list = ['spacy', 'nltk', 'gensim']

for embedding_library, ngram_number in itertools.product(embedding_libraries_list, ngrams_list):
    print(f'Building {embedding_library}_{ngram_number}grams model and vocabulary.')

    for embed_model_name, embed_func_list in embedding_models_dict.items():

        build_train_func, embed_func, model_loader = embed_func_list
        print(f'Building {embed_model_name} from {embed_func.__name__} function.')

        vocab, model = build_train_func(
            df=df_manual,
            ngram_number=ngram_number,
            embedding_library=embedding_library,
        )

        print(f'Getting {embed_model_name} embeddings.')

        df_manual[
            f'Job Description {embedding_library}_{ngram_number}grams_mean_{embed_model_name}_embeddings'
        ] = df_manual[
            f'Job Description {embedding_library}_{ngram_number}grams_original_list'
        ].apply(
            lambda sentences: embed_func(sentences, vocab, model)
        )
        model.save(f'{data_dir}embeddings models/{embedding_library}_{ngram_number}grams_{embed_model_name}_model.model')

    # Sent2Vec
    print('Getting sent2vec embeddings.')
    embeddings_index = get_glove()
    df_manual[f'Job Description {embedding_library}_{ngram_number}grams_sent2vec_embeddings'] = df_manual[f'Job Description {embedding_library}_{ngram_number}grams'].apply(lambda sentences: sent2vec(sentences, embeddings_index=embeddings_index, external_glove=True, extra_preprocessing_enabled=False))
    print('Done getting sent2vec embeddings.')
        
    

In [ ]:
assert len(df_manual) > 0 and isinstance(df_manual, pd.DataFrame)
df_manual.to_pickle(f'{df_save_dir}df_manual_for_trainning.pkl')
df_manual.to_csv(f'{df_save_dir}df_manual_for_trainning.csv', index=False)


# ATTN: This script should be run AFTER all embeddings are completed.


### START HERE IF SOURCING FROM df_manual_FOR_TRAINNING
### PLEASE SET CORRECT DIRECTORY PATHS BELOW


# Descriptives and visualization


In [ ]:
import os
import sys
import importlib
from pathlib import Path
import numpy as np

mod = sys.modules[__name__]

code_dir = None
code_dir_name = 'Code'
unwanted_subdir_name = 'Analysis'

for _ in range(5):

    parent_path = str(Path.cwd().parents[_]).split('/')[-1]

    if (code_dir_name in parent_path) and (unwanted_subdir_name not in parent_path):

        code_dir = str(Path.cwd().parents[_])

        if code_dir is not None:
            break

sys.path.append(code_dir)
# %load_ext autoreload
# %autoreload 2


In [ ]:
from setup_module.imports import *


In [ ]:
def df_gender_age_info(df, ivs_all=None):
    if ivs_all is None:
        ivs_all = [
            'Gender',
            'Gender_Num',
            'Gender_Female',
            'Gender_Mixed',
            'Gender_Male',
            'Age',
            'Age_Num',
            'Age_Older',
            'Age_Mixed',
            'Age_Younger',
        ]
    # Print Info
    print('\nDF INFO:\n')
    df.info()

    for iv in ivs_all:
        try:
            counts = df[f'{iv}'].value_counts()
            percentages = df[f'{iv}'].value_counts(normalize=True).mul(100).round(1).astype(float)
            print('='*20)
            print(f'{iv}:')
            print('-'*20)
            print(f'{iv} Counts:\n{counts}')
            print('-'*20)
            print(f'{iv} Percentages:\n{percentages}')

            with contextlib.suppress(Exception):
                mean = df[f"{iv}"].mean().round(2).astype(float)
                sd = df[f"{iv}"].std().round(2).astype(float)
                print('-'*20)
                print(f'{iv} Mean: {mean}')
                print('-'*20)
                print(f'{iv} Standard Deviation: {sd}')

        except Exception:
            print(f'{iv} not available.')

    print('\n')


In [ ]:
# Function to order categories
def categorize_df_gender_age(df, gender_order=None, age_order=None, ivs=None):
    if gender_order is None:
        gender_order = ['Female', 'Mixed Gender', 'Male']
    if age_order is None:
        age_order = ['Older', 'Mixed Age', 'Younger']
    if ivs is None:
        ivs = ['Gender', 'Age']
    # Arrange Categories
    for iv in ivs:
        if iv == 'Gender':
            order = gender_order
        elif iv == 'Age':
            order = age_order
        try:
            df[iv] = df[iv].astype('category').cat.reorder_categories(order, ordered=True)

            df[iv] = pd.Categorical(
                df[iv], categories=order, ordered=True
            )
            df[f'{iv}_Num'] = pd.to_numeric(df[iv].cat.codes).astype('int64')
        except ValueError as e:
            print(e)

    return df


In [ ]:
df_manual = pd.read_pickle(f'{df_save_dir}df_manual_for_trainning.pkl').reset_index(drop=True)


In [ ]:
# All info
analysis_columns = [
    'Warmth',
    'Competence'
]

df_manual = categorize_df_gender_age(df_manual)

df_manual.info()

# Gender and Age info by job ad
print('='*30)
print('Gender and Age info at Sentence Level')
print('-'*30)
df_gender_age_info(df_manual)

# Gender and Age info by job ad
print('='*30)
print('Gender and Age info at Job Advertisement Level')
print('-'*30)
df_gender_age_info(df_manual.groupby(['Job ID']).first())

# Warmth and Competence info by job ad
print('='*30)
print('Warmth and Competence info at Sentence Level')
print('-'*30)
df_gender_age_info(df_manual, ivs_all = analysis_columns)

# Warmth and Competence info by job ad
print('='*30)
print('Warmth and Competence info at Job Advertisement Level')
print('-'*30)
df_gender_age_info(df_manual.groupby(['Job ID']).first(), ivs_all = analysis_columns)


In [ ]:
# Imbalance Ratio
warmth_imbalance_ratio = df_manual['Warmth'].loc[
    df_manual['Warmth'] == 1].count()/df_manual['Warmth'].loc[df_manual['Warmth'] == 0
].count()
competence_imbalance_ratio = df_manual['Competence'].loc[
    df_manual['Competence'] == 1].count()/df_manual['Competence'].loc[df_manual['Competence'] == 0
].count()

all_imbalance_ratio_dict = {
    'Warmth': warmth_imbalance_ratio,
    'Competence': competence_imbalance_ratio
}

print('='*20)
print('Imabalance Ratios')
print('-'*10)
print(f'Warmth IR: {warmth_imbalance_ratio:.2f}')
print(f'Competence IR: {competence_imbalance_ratio:.2f}')
print('='*20)


In [ ]:
# Ploting Warmth and Competence
df_warm_comp_transposed = pd.concat(
    [
        df_manual['Warmth'].value_counts(normalize=True).mul(100).round(2).astype(float).to_frame().T,
        df_manual['Competence'].value_counts(normalize=True).mul(100).round(2).astype(float).to_frame().T,
    ]
)

fig, ax = plt.subplots()
ax.set_title('Training Dataset: Warmth and Competence Sentence Percentages')
ax.set_xlabel('Manually Annotated Sentencence Warmth and Competence Percentage from Total')

df_warm_comp_transposed.plot(
    kind='barh', legend=True, stacked=True, ax=ax, color=['C5', 'C0'],
)
ax.legend(['Absent', 'Present'])

for container in ax.containers:
    labels = [f'{width:.1f}%' for v in container if float(width:= v.get_width())]
    ax.bar_label(container, labels=labels, label_type='center', color='white')
    ax.set_xlabel('% Job Ad Sentences')

for i, tick_label in enumerate(ax.get_ymajorticklabels()):
    ax.annotate(
        f'IR for {tick_label.get_text()} = {all_imbalance_ratio_dict[tick_label.get_text()]:.2f}',
        xy=(48, 0.3+i), ha='center', va='center'
    )

for save_format in ['eps', 'png']:
    fig.savefig(
        f'{plot_save_path}Manually Annotated Warmth and Competence Sentences.{save_format}',
        format=save_format, dpi=3000, bbox_inches='tight'
    )


In [ ]:
# Ploting Gender and Age
df_gender_transposed = df_manual['Gender'].value_counts(normalize=True).mul(100).round(2).astype(float).to_frame().T
df_age_transposed = df_manual['Age'].value_counts(normalize=True).mul(100).round(2).astype(float).to_frame().T

fig, axs = plt.subplots(1, 2)
fig.suptitle('Training Dataset: Gender and Age Sentence Percentages')

df_gender_transposed.plot(
    kind='bar', legend=True, stacked=True, ax=axs[0], color=['C5', 'C2', 'C0']
)
df_age_transposed.plot(
    kind='bar', legend=True, stacked=True, ax=axs[1], color=['C5', 'C2', 'C0']
)

for ax in axs:
    for container in ax.containers:
        labels = [f'{height:.1f}%' for v in container if float(height:= v.get_height())]
        ax.bar_label(container, labels=labels, label_type='center', color='white')
        ax.legend(loc='upper right', fontsize=8)
        ax.set_ylabel('% Job Ad Sentences')

for save_format in ['eps', 'png']:
    fig.savefig(
        f'{plot_save_path}Manually Annotated Gender and Age Sentences.{save_format}',
        format=save_format, dpi=3000, bbox_inches='tight'
    )


In [ ]:
def make_barplot(df, iv, dvs_list):
    fig, ax = plt.subplots()
    fig.suptitle(f'Training Dataset: Warmth and Competence Sentence Percentages per {iv}')

    vars_list = [iv]
    vars_list.extend(dvs_list)

    df_pivot = df[
        vars_list
    ].pivot_table(
        index=iv, values=dvs_list, fill_value=0, aggfunc=lambda x: (100*x.sum())/len(df)
    )

    df_pivot.sort_values(by=iv, ascending=False).plot(kind='barh', legend=True, stacked=True, ax=ax, color=['C0', 'C5'])

    for _ in ax.containers:
        ax.set_xlabel('% Job Ad Sentences')
        ax.set_ylabel(iv)

    for save_format in ['eps', 'png']:
        fig.savefig(
            f'{plot_save_path}Barplot - Manually Annotated {iv} x {dvs_list[0]} and {dvs_list[1]} Sentences.{save_format}',
            format=save_format, dpi=3000, bbox_inches='tight'
        )

In [ ]:
# Make stacked barplots
ivs_list = ['Gender', 'Age', 'Sector']

for iv in ivs_list:
    make_barplot(df_manual, iv=iv, dvs_list=['Warmth', 'Competence'])


In [ ]:
def make_lineplot(df, iv, dv):

    df = categorize_df_gender_age(df)

    title = f'Means of {dv}-related frames in job ads from {iv} segregated sectors'
    data = df.groupby(iv)[dv].agg('mean')

    line_plot = sns.lineplot(
        data=data, marker='o', legend='full'
    )
    line_plot.set(
        title=title,
        xlabel=f'{iv}-dominated Sector',
        ylabel=f'{dv} Mean',
    )
    fig = line_plot.get_figure()

    for save_format in ['eps', 'png']:
        fig.savefig(
            f'{plot_save_path}Line Plot - Manually Annotated {iv} x {dv} Sentences.{save_format}',
            format=save_format, dpi=3000, bbox_inches='tight'
        )
    plt.show()
    plt.clf()
    plt.cla()
    plt.close()


In [ ]:
# Make line plots
for iv, dv in itertools.product(['Gender', 'Age'], ['Warmth', 'Competence']):
    make_lineplot(df_manual, iv=iv, dv=dv)


# ATTN: This script should be run AFTER all visualizations are completed.


### START HERE IF SOURCING FROM df_manual_FOR_TRAINNING
### PLEASE SET CORRECT DIRECTORY PATHS BELOW


# Make descriptive tables


In [ ]:
import os
import sys
import importlib
from pathlib import Path
import numpy as np

mod = sys.modules[__name__]

code_dir = None
code_dir_name = 'Code'
unwanted_subdir_name = 'Analysis'

for _ in range(5):

    parent_path = str(Path.cwd().parents[_]).split('/')[-1]

    if (code_dir_name in parent_path) and (unwanted_subdir_name not in parent_path):

        code_dir = str(Path.cwd().parents[_])

        if code_dir is not None:
            break

sys.path.append(code_dir)
# %load_ext autoreload
# %autoreload 2


In [ ]:
from setup_module.imports import *


In [ ]:
# Function to order categories
def categorize_df_gender_age(df, gender_order=None, age_order=None, ivs=None):
    if gender_order is None:
        gender_order = ['Female', 'Mixed Gender', 'Male']
    if age_order is None:
        age_order = ['Older', 'Mixed Age', 'Younger']
    if ivs is None:
        ivs = ['Gender', 'Age']

    # Arrange Categories
    for iv in ivs:
        if iv == 'Gender':
            order = gender_order
        elif iv == 'Age':
            order = age_order
        try:
            df[iv] = df[iv].astype('category').cat.reorder_categories(order, ordered=True)

            df[iv] = pd.Categorical(
                df[iv], categories=order, ordered=True
            )
            df[f'{iv}_Num'] = pd.to_numeric(df[iv].cat.codes).astype('int64')
        except ValueError as e:
            print(e)

    return df


In [ ]:
df_manual = pd.read_pickle(f'{df_save_dir}df_manual_for_trainning.pkl').reset_index(drop=True)


### Gender and Age tables

In [ ]:
# Function to make descriptives tables
def genage_make_descriptives_table(df, v, level, gender_order=None, age_order=None):

    if gender_order is None:
        gender_order = ['Female', 'Male', 'Mixed Gender']
    if age_order is None:
        age_order = ['Older', 'Younger', 'Mixed Age']
    
    ivs_dict = {'Gender': gender_order, 'Age': age_order}

    df = categorize_df_gender_age(df)

    if level.title() == 'Job Advertisement':
        level_df = df.groupby(['Job ID']).first()
    elif level.title() == 'Sentence':
        level_df = df
    else:
        raise Exception(f'Specified level {level} not in data.')

    if v in list(ivs_dict.keys()):
        cat_dict = ivs_dict
        index = [
            f'{v_cat}-dominated'
            if 'Mixed' not in v_cat
            else
            f'{"-".join(v_cat.split())}'
            for v_cat in cat_dict[v]
        ]
        caption = [
            f'{v}_{v_cat.split()[0]}'
            for v_cat in cat_dict[v]
        ]

    desc_dict = {
        'Sectors': index,
        'n': [
            level_df[v].value_counts()[v_cat]
            for v_cat in cat_dict[v]
        ],
        '%': [
            level_df[v].value_counts(normalize=True).mul(100).round(2).astype(float)[v_cat]
            for v_cat in cat_dict[v]
        ],
        'M': [
            level_df[caption].mean().round(2).astype(float)[i]
            for i in range(len(cat_dict[v]))
        ],
        'S.D.': [
            level_df[caption].std().round(2).astype(float)[i]
            for i in range(len(cat_dict[v]))
        ]
    }

    # Make DF from dict
    df_desc = pd.DataFrame(desc_dict)
    df_desc.set_index('Sectors', inplace=True)

    return df_desc


In [ ]:
df_desc_gender_dict = defaultdict(list)
df_desc_age_dict = defaultdict(list)

variables = ['Gender', 'Age']
levels = ['Job Advertisement', 'Sentence']
data_types = ['Manually Annotated', 'Collected']

for v, level, data_type in itertools.product(
    variables, levels, data_types
):
    cols = [
        (f'{data_type.title()} Job Advertisements', v, level.title(), 'n'),
        (f'{data_type.title()} Job Advertisements', v, level.title(), '%'),
        (f'{data_type.title()} Job Advertisements', v, level.title(), 'M'),
        (f'{data_type.title()} Job Advertisements', v, level.title(), 'S.D.'),
    ]

    df_desc_genage = genage_make_descriptives_table(df=df_manual, v=v, level=level)

    df_desc_genage.columns = pd.MultiIndex.from_tuples(cols)

    if v == 'Gender':
        df_desc_gender_dict[data_type].append(df_desc_genage)
    elif v == 'Age':
        df_desc_age_dict[data_type].append(df_desc_genage)


In [ ]:
df_desc_genage_dict = defaultdict(list)

for df, data_type in itertools.product(
    [df_desc_gender_dict, df_desc_age_dict],
    data_types,
):
    category = df[data_type][0].columns.get_level_values(level=1)[0]
    if df[data_type][0].columns.get_level_values(level=2).str.contains('Job Advertisement').all():
        df_desc_genage_dict[category].append(
            pd.concat(
                [
                    df[data_type][0], df[data_type][1]
                ],
                axis='columns'
            )
        )
    else:
        df_desc_genage_dict[category].append(
            pd.concat(
                [
                    df[data_type][1], df[data_type][0]
                ],
                axis='columns'
            )
        )


In [ ]:
for cat, df in df_desc_genage_dict.items():
    df = pd.concat([df[0], df[1]], axis='columns')

    if cat == 'Gender':
        df_desc_gender = df
    elif cat == 'Age':
        df_desc_age = df


In [ ]:
# Save Tables
# Gender
df_desc_gender.to_csv(f'{table_save_path}Gender - Job Advertisement Descriptives.csv', index=True)
df_desc_gender.to_pickle(f'{table_save_path}Gender - Job Advertisement Descriptives.pkl')
with pd.option_context('max_colwidth', 10000000000):
    df_desc_gender.style.to_latex(f'{table_save_path}Gender - Job Advertisement Descriptives.tex', index=True, longtable=True, escape=True, multicolumn=True, multicolumn_format='c', position='H', caption='Number and proportion of job advertisements sample, and resulting sentences for gender and age homogeneous and heterogeneous sectors', label='Descriptives')
df_desc_gender.to_markdown(f'{table_save_path}Gender - Job Advertisement Descriptives.md', index=True)
# save_sector_excel(df_sectors_all, data_save_dir)

# Age
df_desc_age.to_csv(f'{table_save_path}Age - Job Advertisement Descriptives.csv', index=True)
df_desc_age.to_pickle(f'{table_save_path}Age - Job Advertisement Descriptives.pkl')
with pd.option_context('max_colwidth', 10000000000):
    df_desc_age.style.to_latex(f'{table_save_path}Age - Job Advertisement Descriptives.tex', index=True, longtable=True, escape=True, multicolumn=True, multicolumn_format='c', position='H', caption='Number and proportion of job advertisements sample, and resulting sentences for gender and age homogeneous and heterogeneous sectors', label='Descriptives')
df_desc_age.to_markdown(f'{table_save_path}Age - Job Advertisement Descriptives.md', index=True)
# save_sector_excel(df_sectors_all, data_save_dir)


### Warmth and Competence tables


In [ ]:
variables = ['Warmth', 'Competence']

cols = [
    ('Manually Annotated Job Advertisements', 'n'),
    ('Manually Annotated Job Advertisements', '%'),
    ('Manually Annotated Job Advertisements', 'M'),
    ('Manually Annotated Job Advertisements', 'S.D.'),
]

binary_order = [0, 1]
cat_dict = {'Warmth': binary_order, 'Competence': binary_order}

desc_warmcomp_dict = {
    'Frames': list(cat_dict.keys()),
    'n': [
        df_manual[v].value_counts()[1]
        for v in variables
    ],
    '%': [
        df_manual[v].value_counts(normalize=True).mul(100).round(2).astype(float)[1]
        for v in variables
    ],
    'M': list(
        df_manual[variables].mean().round(2).astype(float)
    ),
    'S.D.': list(
        df_manual[variables].std().round(2).astype(float)
    )
}
    
# Make df_manual from dict
df_desc_warmcomp = pd.DataFrame(desc_warmcomp_dict)
df_desc_warmcomp.set_index('Frames', inplace=True)
df_desc_warmcomp.columns = pd.MultiIndex.from_tuples(cols)


In [ ]:
# Save Tables
df_desc_warmcomp.to_csv(f'{table_save_path}Warmth and Competence - Job Advertisement Descriptives.csv', index=True)
df_desc_warmcomp.to_pickle(f'{table_save_path}Warmth and Competence - Job Advertisement Descriptives.pkl')
with pd.option_context('max_colwidth', 10000000000):
    df_desc_warmcomp.style.to_latex(f'{table_save_path}Warmth and Competence - Job Advertisement Descriptives.tex', index=True, longtable=True, escape=True, multicolumn=True, multicolumn_format='c', position='H', caption='Number and proportion of job advertisements sample, and resulting sentences for gender and age homogeneous and heterogeneous sectors', label='Descriptives')
df_desc_warmcomp.to_markdown(f'{table_save_path}Warmth and Competence - Job Advertisement Descriptives.md', index=True)
# save_sector_excel(df_sectors_all, data_save_dir)
